In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# <p style="background-color:#80ccff; font-family:newtimeroman; font-size:150%; text-align:center; border-radius:  80px 5px; padding-top:8px; padding-bottom:8px;">Input</p>

In [1]:
treino = pd.read_csv('../input/emnist/emnist-letters-train.csv')

In [1]:
x = np.array(treino.iloc[:,1:].values)
y = np.array(treino.iloc[:,0].values)

In [1]:
fig,axes = plt.subplots(3,5,figsize=(10,8))
for i,ax in enumerate(axes.flat):
    ax.imshow(x[i].reshape([28,28]))

# <p style="background-color:#80ccff; font-family:newtimeroman; font-size:150%; text-align:center; border-radius:  80px 5px; padding-top:8px; padding-bottom:8px;">Normalize and Reshaping</p>

In [1]:
x = x/255

treino_shape = treino.shape[0]
treino_height = 28
treino_width = 28
treino_size = treino_height*treino_width

x = x.reshape(treino_shape, treino_height, treino_width, 1)

# <p style="background-color:#80ccff; font-family:newtimeroman; font-size:150%; text-align:center; border-radius:  80px 5px; padding-top:8px; padding-bottom:8px;">Number of Class</p>

In [1]:
classes = len(treino['45'].unique())

In [1]:
y = to_categorical(y, classes)

In [1]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=.15, stratify=y)
print(x_treino.shape, y_treino.shape)
print(x_teste.shape, y_teste.shape)

# <p style="background-color:#80ccff; font-family:newtimeroman; font-size:150%; text-align:center; border-radius:  80px 5px; padding-top:8px; padding-bottom:8px;">Model</p>

In [1]:
'''modelo = Sequential()
modelo.add(Conv2D(32, 3, input_shape=(28,28,1)))
modelo.add(MaxPool2D(2,2))
modelo.add(Flatten(input_shape=(28,28,1)))
modelo.add(Dense(512, activation='relu'))
modelo.add(Dense(128, activation='relu'))
modelo.add(Dense(classes, activation='softmax'))
modelo.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])'''

In [1]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(classes, activation = "softmax"))

model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

# <p style="background-color:#80ccff; font-family:newtimeroman; font-size:150%; text-align:center; border-radius:  80px 5px; padding-top:8px; padding-bottom:8px;">Callback</p>

In [1]:
parada = EarlyStopping(monitor='val_accuracy', min_delta=0, verbose=0, restore_best_weights=True, patience=3,
                      mode='max')
redutor = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2, mil_lr=0.0001)

In [1]:

history = model.fit(x_treino, y_treino, batch_size=64,
                              epochs = 20, validation_data = (x_teste, y_teste),
                              verbose = 1, steps_per_epoch=x_treino.shape[0] // 64
                              , callbacks=[redutor,parada])

In [1]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
axes[0].plot(history.history['accuracy'])
axes[0].plot(history.history['val_accuracy'])
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Acurácia')
axes[0].legend(['Acurácia em Treino','Acurácia em Teste'])
axes[0].grid(True)

axes[1].plot(history.history['loss'])
axes[1].plot(history.history['val_loss'])
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Erro')
axes[1].legend(['Erro em Treino','Erro em Teste'])
axes[1].grid(True)

# <p style="background-color:#80ccff; font-family:newtimeroman; font-size:150%; text-align:center; border-radius:  80px 5px; padding-top:8px; padding-bottom:8px;">Augmentation</p>

In [1]:
aug = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,
        rotation_range=3,  # Gira a imagem em 3 graus
        zoom_range = 0.1, # Da um zoom na imagem de forma aleatória
        width_shift_range=0.1,  # Move a imagem de forma aleatória horizontalmente
        height_shift_range=0.1)  # Move a imagem de forma aleatória verticalmente
aug.fit(x_treino)

In [1]:
history = model.fit(aug.flow(x_treino, y_treino, batch_size=64),
                              epochs = 20, validation_data = (x_teste, y_teste),
                              verbose = 1, steps_per_epoch=x_treino.shape[0] // 64
                              , callbacks=[redutor,parada])

# <p style="background-color:#80ccff; font-family:newtimeroman; font-size:150%; text-align:center; border-radius:  80px 5px; padding-top:8px; padding-bottom:8px;">Result</p>

In [1]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
axes[0].plot(history.history['accuracy'])
axes[0].plot(history.history['val_accuracy'])
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Acurácia')
axes[0].legend(['Acurácia em Treino','Acurácia em Teste'])
axes[0].grid(True)

axes[1].plot(history.history['loss'])
axes[1].plot(history.history['val_loss'])
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Erro')
axes[1].legend(['Erro em Treino','Erro em Teste'])
axes[1].grid(True)